In [28]:
from langchain.chat_models import ChatOllama, ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, load_prompt
from langchain.schema import BaseOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain_core.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.prompts.prompt import PromptTemplate


In [29]:
llm = ChatOllama(
    model = "llama3:8b",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [30]:
example_answer = [
{
"question": """
아래는 대학영어 강의에 대한 강의평이야. 강의평을 요약해.
1. 정말 좋은 교수님과 너무 고등학생 같은 시험.. 꼼꼼히 공부해야 합니다!
2. 설명 잘 해주시고 시험은 내신처럼 나옵니다. 꼼꼼히 지문 보세요.
3. 설명 잘해주시고 고등학교 영어 내신처럼 공부하면 됩니다
4. 교수님께서 설명을 친절하게 해주십니다. 2인1조로 구성되어 매수업 해야할 과제가 있지만 강의시간 안에 끝나서 부담이 없습니다. 정말 좋은 교수님이라 추천합니다
5. 기출문제 주십니다. 매 수업 팀 과제가 귀찮아요.  (2인 1조) 쁠은 다 채워주셔요.
""",
"answer": """
시험은 고등학교 내신처럼 나오며 매 수업 팀 과제가 있지만 교수님이 매우 좋아요.
""",
},
{
"question": """
아래는 광운인되기 강의에 대한 강의평이야. 강의평을 요약해.
1. 강의만 온라인으로 다 들으면 pass주는 과목입니다. 1학점 챙겨가세요
2. 온라인강의 꿀! 학기 말에 몰아서 하면 됩니다!!
3. 강의만 수강하시면 패스 충분히 받습니다. 온라인 강의라 부담이 없습니다
4. 강의 제때제때 틀어놓는 거랑 공지사항 내용 꼼꼼히 확인하기만 하면 쉽게 패스 가능합니다
""",
"answer": """
공지사항을 제때 확인해야 하지만, 온라인 강의라 부담이 없어요.
""",
},
{
"question": """
아래는 유기화학1 강의에 대한 강의평이야. 강의평을 요약해.
1. 교수님 강의력 좋은 편은 아님.. 족보 타는 강의 기말은 많이는 안탄듯 가지고 있으면 좋음
2. 족보 너무 많이 탑니다 ㅠ 족보 무조건 구해서 시험 보시는 거 추천드립니다
3. 과제, 족보 이 두개만 풀면 시험 무난하게 볼 수 있어요. 근데 성적을 잘 주시는 편은 아닌 것 같습니다
""",
"answer": """
교수님의 강의력이 좋은 편이 아니고, 과제와 족보가 중요하며 성적을 잘 주시는 편이 아니예요.
""",
},
]

In [32]:
import sqlite3

conn = sqlite3.connect('../database/lecture.db')
cursor = conn.cursor()

cursor.execute('SELECT lecNumber, lecClassName FROM LectureTable')
lectures = cursor.fetchall()

for lecture in lectures:
    if len(lecture) < 2:
        print("lecNum or lecClassName is null")
        continue
    
    lecNumber, lecClassName = lecture

    cursor.execute('''
        SELECT lecReview 
        FROM LecReviewTable 
        WHERE lecNumber = ?
    ''', (lecNumber,))
    
    reviews = cursor.fetchall()
    
    if len(reviews) == 0:
        print(f"{lecNumber}'s review is null")
        continue
    
    formatted_reviews = []
    for i, review in enumerate(reviews[:5]): # 상위 5개만
        formatted_review = review[0].replace('\n', ' ')
        formatted_reviews.append(f"{i+1}. {formatted_review}")
    
    review_texts = "\n".join(formatted_reviews)
    
    lecture_info = {
        "lecNumber": lecNumber,
        "lecClassName": lecClassName,
        "lecReview": f"\n{review_texts}\n"
    }
    
    question = f"""
아래는 {lecClassName} 강의에 대한 강의평이야. 강의평을 요약해.
{review_texts}
    """
    
    example_prompt = ChatPromptTemplate.from_messages(
        [
            ("human", "{question}"),
            ("ai", "{answer}"),
        ]
    )

    example_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt,
        examples=example_answer,
    )

    final_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "너는 강의를 선택하는 학생들을 위해 강의 리뷰를 한 줄로 요약하는 봇이야."),
            example_prompt,
            ("human", "강의 리뷰 리스트들을 함축해서 최종적으로 한 줄의 답변만을 내놓아야 해. 리뷰에서 중복되거나 필수적이라고 생각되는 부분만을 한국어로 한 줄로 요약해. 요약 외에는 어떠한 답변도 더하지 마.\n"+question),
        ]
    )

    chain = final_prompt | llm

    answer = chain.invoke({"question": question})

    final_answer = answer.content

    cursor.execute('''
        UPDATE LectureTable
        SET lecSummaryReview = ?
        WHERE lecNumber = ?
    ''', (final_answer, lecNumber))
    conn.commit()
    
    # print(f"\n{lecNumber} {lecClassName} answer: {final_answer}")

conn.close()



강의만 온라인으로 수강하면 패스 충분히 받을 수 있습니다. 강의 시청기간과 과제를 잘 지켜야 합니다.
(Note: I summarized the reviews into a single sentence, focusing on the main points and omitting repetitive or minor details.)
출석만 제때 하면 되고, 강의는 P/NP라서 쉽게 학점을 채울 수 있습니다.

고등학교 영어 내신 준비하는 기분으로 강의가 진행되며, 교재 열심히 보시면 중간, 기말 다 잘 보실 수 있습니다.

교수님이 친절하고 이름도 기억해주신다. 과제는 많지만, 온라인으로 빼주는 날이 가장 좋은 것 같다.
(Note: I tried to summarize the reviews in a single sentence while keeping the main points and tone of the original reviews)
강의는 교재 읽어주시는 것과 글쓰기 과제가 주요 내용으로, 실제로는 공부할 것이 거의 없으며 시험 문제도 기존 교재 내용과 다르게 나왔습니다.
고등학교 영어 내신 준비하는 기분으로 강의를 진행하며, 교재 열심히 보시면 중간, 기말 다 잘 보실 수 있습니다.
(Note: I summarized the reviews into a single sentence that captures the overall tone and content of the reviews. The original reviews were quite similar in their descriptions of the course, so the summary is not too different from any one of them.)
교수님이 친절하고 과제가 적으며 시험은 쉽지만 강의력과 강의 내용은 딱히 들을 게 없음.

KeyboardInterrupt: 